In [1]:
import torch
import random

random.seed(42)
torch.manual_seed(42)
#torch.set_default_device('mps')

In [2]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1

### Load the model

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_path = "microsoft/Phi-3-mini-4k-instruct"
model_path = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_path, legacy=True, use_fast=True, add_eos_token=True, max_length=250)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Model Configuration

In [4]:
from peft import LoraConfig, PeftModel

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=[
          'q_proj',
          'k_proj',
          'v_proj',
          'o_proj'
        ]
)

model = PeftModel(base_model, lora_config, "mistral7Bv01-results/checkpoint-100")

device = torch.device("mps")
model.to(device)

/Users/chabhara/miniforge3/envs/DSE/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (mistral7Bv01-results/checkpoint-100): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (mistral7Bv01-results/checkpoint-100): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (mistral7Bv01-results/checkpoint-100): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora

### Load the dataset

In [16]:
from datasets import load_dataset
from langchain_core.prompts import PromptTemplate

dataset = load_dataset("b-mc2/sql-create-context")
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

In [ ]:
#dataset['train'].to_pandas().head(50).to_html()

### Preprocess dataset for the need

In [25]:
def create_prompt(sample):
    prompt_template = PromptTemplate.from_template("""Generate an SQL query using the following database schema:
{database_schema}, to answer the following question:
{user_question}.
Response: <s>{user_response}</s>""")
    user_message = sample['question']
    user_response = sample['answer']
    database_schema = sample['context']

    template = prompt_template.format(database_schema=database_schema,user_question=user_message,user_response=user_response)
    return {"inputs":template}


instruct_tune_dataset = dataset.map(create_prompt)
print(instruct_tune_dataset)

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context', 'inputs'],
        num_rows: 78577
    })
})


In [6]:
# test = {
#     "question": "How many heads of the departments are older than 56 ?",
#     "answer" : "SELECT COUNT(*) FROM head WHERE age > 56",
#     "context" : "CREATE TABLE head (age INTEGER)"
# }
# create_prompt(test)

In [33]:
instruct_tune_dataset['train'].to_pandas().head(5)

,answer,question,context,inputs
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),Generate an SQL query using the following data...
1,"SELECT name, born_state, age FROM head ORDER B...","List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...",Generate an SQL query using the following data...
2,"SELECT creation, name, budget_in_billions FROM...","List the creation year, name and budget of eac...","CREATE TABLE department (creation VARCHAR, nam...",Generate an SQL query using the following data...
3,"SELECT MAX(budget_in_billions), MIN(budget_in_...",What are the maximum and minimum budget of the...,CREATE TABLE department (budget_in_billions IN...,Generate an SQL query using the following data...
4,SELECT AVG(num_employees) FROM department WHER...,What is the average number of employees of the...,CREATE TABLE department (num_employees INTEGER...,Generate an SQL query using the following data...


In [8]:
train_dataset = instruct_tune_dataset.map(batched=True,remove_columns=['answer', 'question', 'context'])
train_dataset

DatasetDict({
    train: Dataset({
        features: ['inputs'],
        num_rows: 78577
    })
})

In [5]:
import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.9"
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

### Training Parameters

In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./mistral7Bv01-results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    auto_find_batch_size = True,
    #per_device_train_batch_size=4,
    # optim = "paged_adamw_32bit",
    gradient_accumulation_steps=1,
    lr_scheduler_type = "cosine",
    warmup_ratio = 0.03,
    #eval_strategy='no',
    save_strategy='steps',
    learning_rate=3e-4,
    eval_strategy = "steps",
    prediction_loss_only = True,
    bf16=True,
    max_steps=100,
    #warmup_steps=50,
    seed=42,
    #eval_steps=50
)

### Training

In [11]:
from transformers import DataCollatorForLanguageModeling
from trl import SFTTrainer

trainer = SFTTrainer(
            model=model,
            train_dataset=train_dataset["train"],
            #eval_dataset=dataset['test'],
            dataset_text_field="inputs",
            tokenizer=tokenizer,
            args=training_args,
            packing=False,
            max_seq_length=500
)

[2024-09-14 21:07:50,619] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to mps (auto detect)


W0914 21:07:51.069116 8194789184 torch/distributed/elastic/multiprocessing/redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/chabhara/miniforge3/envs/DSE/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/chabhara/miniforge3/envs/DSE/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/Users/chabhara/miniforge3/envs/DSE/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will ov

In [12]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=100, training_loss=0.9979658508300782, metrics={'train_runtime': 7963.5804, 'train_samples_per_second': 0.1, 'train_steps_per_second': 0.013, 'total_flos': 4406573824868352.0, 'train_loss': 0.9979658508300782, 'epoch': 0.010180189351521938})

In [5]:
def create_prompt_for_test(sample):
    prompt_template = PromptTemplate.from_template("""Generate an SQL query using the following database schema:
{database_schema}, to answer the following question:
{user_question}. Also, suggest a chart type best suited to plot the data generated by the query.
Response:""")
    user_message = sample['question']
    #user_response = sample['answer']
    database_schema = sample['context']

    template = prompt_template.format(database_schema=database_schema,user_question=user_message)
    return template

---
### Output
---

In [11]:
from builtins import input

#question = input("Please enter your query:Start your query likeGenerate an SQL query to retrieve...")
question = "How many days had both mean humidity above 50 and mean visibility above 8?"

In [12]:
#database_schema = input("Please enter database schema:CREATE TABLE tablename (columns <datatype)")
database_schema = "CREATE TABLE weather (mean_humidity VARCHAR, mean_visibility_miles VARCHAR)"

In [13]:
from langchain_core.prompts import PromptTemplate

In [14]:
test_query = {
    "question": question,
    "context" : database_schema
}


model_inputs = tokenizer(create_prompt_for_test(test_query), return_tensors="pt").to("mps:0")

output = model.generate(**model_inputs, max_length=500, no_repeat_ngram_size=30, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)[0]

result = tokenizer.decode(output, skip_special_tokens=False)

In [15]:
from IPython.display import display, Markdown

result = result.replace('<s>','').replace('</s>','')
display(Markdown(result))

 Generate an SQL query using the following database schema:
CREATE TABLE weather (mean_humidity VARCHAR, mean_visibility_miles VARCHAR), to answer the following question:
How many days had both mean humidity above 50 and mean visibility above 8?. Also, suggest a chart type best suited to plot the data generated by the query.
Response:
```
SELECT COUNT(*) AS days
FROM weather
WHERE mean_humidity > '50' AND mean_visibility_miles > '8'
```
The best chart type to plot the data generated by this query would be a line chart, as it would allow for easy visualization of the trend in the number of days with both mean humidity above 50 and mean visibility above 8 over time.

<hr>